In [1]:
import os

from typing import Any, List, Dict, Mapping, Tuple, Union, Optional
import copy
from dataclasses import dataclass, field
import json
import pathlib
from typing import Dict, Optional, Sequence
import pickle
import tqdm

import numpy as np
import torch
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset
import transformers
from transformers import Trainer
from transformers.trainer_pt_utils import LabelSmoother

from transformers import T5ForConditionalGeneration, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")

In [3]:
with open("../finetune_data_prepared.jsonl") as f:
    data = [json.loads(line) for line in f]

In [7]:
all_examples = []

for item in tqdm.tqdm(data):
    prompt = item["prompt"]
    answer = item["completion"].strip()

    prompt_ids = tokenizer.encode(prompt, add_special_tokens=False)
    answer_ids = tokenizer.encode(answer, add_special_tokens=False) + [tokenizer.eos_token_id]

    input_ids = torch.tensor(prompt_ids, dtype=torch.long)
    answer_ids = torch.tensor(answer_ids, dtype=torch.long)

    if len(input_ids) > 2048:
        print("skipping", len(input_ids))
        continue

    # labels = input_ids.clone()
    # labels[: len(prompt_ids)] = -100

    all_examples.append(
        dict(
            input_ids=input_ids,
            labels=answer_ids,
            attention_mask=torch.ones_like(input_ids, dtype=torch.bool),
        )
    )

  2%|▏         | 117/6203 [00:00<00:15, 388.07it/s]

skipping 2196


  6%|▌         | 373/6203 [00:00<00:14, 411.52it/s]

skipping 2116


  8%|▊         | 498/6203 [00:01<00:14, 397.91it/s]

...

 97%|█████████▋| 6016/6203 [00:15<00:00, 345.92it/s]

skipping 2168


 99%|█████████▉| 6166/6203 [00:15<00:00, 363.87it/s]

skipping 2166


100%|██████████| 6203/6203 [00:16<00:00, 386.60it/s]


In [9]:
with open("./t5_finetune_dataset.pkl", "wb") as f:
    pickle.dump(all_examples, f)

In [10]:
[1,2,3][:-1]

[1, 2]